<a href="https://colab.research.google.com/github/rk119/F20BC/blob/main/F20BC_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Imports

In [2]:
import numpy as np
import pandas as pd

### Load the data

In [6]:
df = x_train_all = pd.read_csv("/content/drive/MyDrive/F20BC-Dataset/data_banknote_authentication.csv")

### View the first 5 instances of the data

In [7]:
df.head()

,3.6216,8.6661,-2.8073,-0.44699,0
0,4.54590,8.1674,-2.4586,-1.46210,0
1,3.86600,-2.6383,1.9242,0.10645,0
2,3.45660,9.5228,-4.0112,-3.59440,0
3,0.32924,-4.4552,4.5718,-0.98880,0
4,4.36840,9.6718,-3.9606,-3.16250,0


### Activation Functions

In [13]:
class ActivationFunctions:
  def evaluate(self,x):
    pass
  def derivate(self,x):
    pass

class Identity:
  def evaluate(self,x):
    return x
  def derivative(self,x):
    return 1

class Sigmoid(ActivationFunctions):
  def evaluate(self,x):
    return 1 / (1 + np.exp(-x))
  def derivative(self,x):
    f = self.evaluate(x)
    return f * (1-f)

class Tanh(ActivationFunctions):
  def evaluate(self,x):
    return np.tanh(x)
  def derivative(self,x):
    f = self.evaluate(x)
    return 1 - f ** 2

class relu(ActivationFunctions):
  def evaluate(self,x):
    return np.maximum(0, x)
  def derivative(self,x):
    return (x > 0).astype(float)

### Loss Functions

In [16]:
class LossFunctions:
  def evaluate(self,x):
    pass
  def derivate(self,x):
    pass

# y is predictions made by the neural network
# t is target/actual numbers corresponding to inputs
class Mse(LossFunctions):
  def evaluate(self, y, t):
    return ((t - y) ** 2).mean()
  def derivative(self, y, t):
    return 2 * (y - t) / len(y)

class BinaryCrossEntropy(LossFunctions):
  def evaluate(self, y, t):
    y_pred = np.clip(y, 1e-7, 1 - 1e-7)
    term0 = (1 - t) * np.log(1 - y_pred + 1e-7)
    term1 = t * np.log(y_pred + 1e-7)
    return - (term0 + term1).mean()

  def derivative(self, y, t):
    y_pred = np.clip(y, 1e-7, 1 - 1e-7)
    return (t / y_pred) - (1 - t) / (1 - y_pred)

class Hinge(LossFunctions):
  def evaluate(self, y, t):
    return np.maximum(0, 1 - t * y).mean()

  def derivative(self, y, t):
    return np.where(t * y < 1, -t, 0)

In [21]:
# Define test cases
test_cases = [
    {"y": np.array([1,2,3,4,5]), "t": np.array([1,2,3,5,5]), "expected": 0.2},
    {"y": np.array([1, 2, 3]), "t": np.array([2, 3, 4]), "expected": 1.0},
    # Add more test cases as needed
]

# Test function
def test_MSE_loss(loss_function, test_cases):
    for i, test_case in enumerate(test_cases, start=1):
        y, t = test_case["y"], test_case["t"]
        expected = test_case["expected"]
        result = loss_function.evaluate(y, t)
        assert np.isclose(result, expected), f"Test case {i} failed: expected {expected}, got {result}"
        print(f"Test case {i} passed.")

test_MSE_loss(Mse(), test_cases)

Test case 1 passed.
Test case 2 passed.


### Layer Initialization

In [22]:
class Layer:
  def __init__(self, input_size, nodes, activation):
    self.nb_nodes = nodes
    self.X_in = None
    self.W = np.random.randn(input_size, nodes)
    self.B = np.random.randn(nodes)
    self.activation = activation

  def forward(self, input_data):
    self.X_in = input_data
    z = np.dot(input_data, self.W) + self.B
    out = self.activation.evaluate(z)
    return out

### Neural Network

In [23]:
class NeuralNetwork:
  def __init__(self):
    self.layers = []

  def add(self, layer):
    self.layers.append(layer)

  def predict(self, input_data):
    output = input_data
    for layer in self.layers:
      output = layer.forward(output)
    return output

  def calculate_loss(self, y_pred, y_true, loss_function):
    return loss_function.evaluate(y_pred, y_true)

In [9]:
# class NeuralNetwork:
#     def __init__(self, layer_sizes):
#         self.layer_sizes = layer_sizes
#         self.weights = []
#         self.biases = []

#         # Initialize weights and biases
#         for i in range(len(layer_sizes) - 1):
#             self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]))
#             self.biases.append(np.random.randn(layer_sizes[i+1]))

#     def forward(self, x, activation_func=ActivationFunctions.logistic):
#         for w, b in zip(self.weights, self.biases):
#             x = activation_func(np.dot(x, w) + b)
#         return x

In [10]:
# # Example usage:
# nn = NeuralNetwork([2, 3, 1])
# input_data = np.array([0.5, 0.25])
# output = nn.forward(input_data, activation_func=ActivationFunctions.logistic)
# print(output)

[0.11984044]
